In [1]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import country_converter as coco

import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
from PIL import Image
import requests

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
books = pd.read_csv("../data/Books.csv")
ratings = pd.read_csv("../data/Ratings.csv")
users = pd.read_csv("../data/Users.csv")

In [3]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [4]:
ratings.columns


Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [5]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [8]:
import pymysql
import pandas as pd

# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE = "bookrecommendation"

# Connect to the MySQL database
try:
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    print("Connected to MySQL Database")

    cursor = connection.cursor()

    # Function to insert data into a table
    def insert_data_from_csv(table_name, csv_file, columns):
        # Load the CSV file with explicit data types
        data = pd.read_csv(csv_file, dtype=str)  # Load everything as string to avoid type issues
        data = data.fillna('NULL')  # Replace NaN values with 'NULL' for MySQL compatibility
        
        # Prepare the INSERT query with the IGNORE keyword to skip duplicates
        placeholders = ", ".join(["%s"] * len(columns))
        query = f"INSERT IGNORE INTO {table_name} ({ ', '.join(columns) }) VALUES ({placeholders})"
        
        # Insert each row into the table
        for _, row in data.iterrows():
            try:
                cursor.execute(query, tuple(row))
            except pymysql.Error as e:
                print(f"Skipping duplicate row in {table_name}: {e}")
        
        connection.commit()
        print(f"Data inserted into {table_name} successfully")

    # Insert Books data (corrected to lowercase 'books' and handling duplicates)
    insert_data_from_csv(
        table_name="books",  # Corrected to lowercase 'books'
        csv_file="../data/Books.csv",  # Path to Books.csv
        columns=['ISBN', 'Book_Title', 'Book_Author', 'Year_Of_Publication', 'Publisher',
                 'Image_URL_S', 'Image_URL_M', 'Image_URL_L']
    )

    # Insert Ratings data (corrected to lowercase 'ratings' and handling duplicates)
    insert_data_from_csv(
        table_name="ratings",  # Corrected to lowercase 'ratings'
        csv_file="../data/Ratings.csv",  # Path to Ratings.csv
        columns=['User_ID', 'ISBN', 'Book_Rating']
    )

    # Insert Users data (corrected to lowercase 'users' and handling duplicates)
    insert_data_from_csv(
        table_name="users",  # Corrected to lowercase 'users'
        csv_file="../data/Users.csv",  # Path to Users.csv
        columns=['User_ID', 'Location', 'Age']
    )

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Database
Data inserted into books successfully
Data inserted into ratings successfully
Data inserted into users successfully
MySQL connection is closed


In [10]:
import pymysql
import pandas as pd

# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE = "bookrecommendation"

# Connect to the MySQL database
try:
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    print("Connected to MySQL Database")

    cursor = connection.cursor()

    # Function to insert ratings data into the 'ratings' table
    def insert_ratings_from_csv(csv_file, columns):
        # Load the CSV file with explicit data types
        data = pd.read_csv(csv_file, dtype=str)  # Load everything as string to avoid type issues
        data = data.fillna('NULL')  # Replace NaN values with 'NULL' for MySQL compatibility
        
        # Prepare the INSERT query with the IGNORE keyword to skip duplicates
        columns = [col for col in columns]  # Ensure we are working with the correct columns
        placeholders = ", ".join(["%s"] * len(columns))
        query = f"INSERT IGNORE INTO ratings ({ ', '.join(columns) }) VALUES ({placeholders})"
        
        # Insert each row into the table
        for _, row in data.iterrows():
            try:
                cursor.execute(query, tuple(row))
            except pymysql.Error as e:
                print(f"Skipping duplicate row: {e}")
        
        connection.commit()
        print("Ratings data inserted successfully")

    # Insert Ratings data (corrected to lowercase 'ratings' and handling duplicates)
    insert_ratings_from_csv(
        csv_file="../data/Ratings.csv",
        columns=['User_ID', 'ISBN', 'Book_Rating']  # Columns to insert
    )

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Database
Skipping duplicate row: (1206, 'The total number of locks exceeds the lock table size')
Skipping duplicate row: (1206, 'The total number of locks exceeds the lock table size')
Ratings data inserted successfully
MySQL connection is closed
